# Raspagem de dados - Aplicação num site de Xadrez

## Introdução

Neste notebook vou treinar como fazer raspagem de dados no site da house of chess para obter uma tabela contendo o nome do conjunto das peças, preço, seu id, tipo de madeira e altura do rei, que é por padrão o parâmetro para saber o tamanho do conjunto em geral. 

Importando as bibliotecas iniciais

In [2]:
from requests import get
from bs4 import BeautifulSoup
from pandas import DataFrame

O site onde está o catálogo das peças de xadrez de plástico é o que está abaixo.

In [3]:
url = 'https://www.houseofchess.com/wood-chess-pieces.html'

Para acessá-lo, poderíamos simplesmente usar a função ```get``` da biblioteca requests, mas o nosso endereço de IP pode ser bloqueado se não mudarmos seu cabeçalho, que está como ```python-requests``` por padrão (esse cabeçalho é interpretado pelos sites como se um bot quisesse acessá-lo). 

Podemos alterá-lo digitando o seguinte:

In [4]:
cabecalho = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

Assim, o site vai "achar" que é um humano que o está acessando.

Feito isso, vamos agora entrar de fato no site.

In [5]:
pagina = get(url, headers = cabecalho)

Vejamos qual é o tipo da variável ```pagina```.

In [22]:
type(pagina)

requests.models.Response

## Coleta de dados

A string correspondente ao código em html do site está no atributo ```pagina.text```, que passaremos como argumento no BeautifulSoup.

In [6]:
sopa = BeautifulSoup(pagina.text, 'html.parser')

Observando o código fonte da página, podemos nos atentar à tag ```div``` cujo atributo é ```class='item_sel_wrapper'```, que contém todas as informações que queremos coletar.

In [7]:
tag = sopa.find(name = 'div', class_='item_sel_wrapper')

### Código do Produto

Começando pelo código do produto, ele se encontra na tag ```div``` com atributo ```class_='item_sel_outer_mid_desc_code'```.

In [8]:
tag.find(name = 'div', class_='item_sel_outer_mid_desc_code').string

'Item# CP2485'

Fazendo isso para todos os produtos, temos

In [9]:
ListaCodigos = []

for subtag in tag.find_all(name = 'div', class_ = 'item_sel_outer_mid_desc_code'):
    ListaCodigos.append(subtag.string)
    
ListaCodigos

['Item# CP2485',
 'Item# CP1774',
 'Item# CP3874',
 'Item# CP2358',
 'Item# CP2810',
 'Item# CP7629',
 'Item# CP9825',
 'Item# CP7695',
 'Item# CP8278',
 'Item# CP6872',
 'Item# CP6725',
 'Item# CP3320',
 'Item# CP5113',
 'Item# CP6505',
 'Item# CP9769',
 'Item# CP1478',
 'Item# CO6780',
 'Item# CP4098',
 'Item# CP5760',
 'Item# CP6964',
 'Item# CP8657',
 'Item# CP8623',
 'Item# CO8065',
 'Item# CP2412',
 'Item# CP3943',
 'Item# CP2793',
 'Item# CP3167',
 'Item# CP4905',
 'Item# CP8544',
 'Item# CP1097',
 'Item# CP4685',
 'Item# CP9759',
 'Item# CP6687',
 'Item# CP8474',
 'Item# 554-76-EBZ',
 'Item# CP7172',
 'Item# CP7491',
 'Item# CP2496',
 'Item# 554-76-RRW-LSB',
 'Item# CP8596',
 'Item# CP1767',
 'Item# CP2372',
 'Item# CP8534',
 'Item# CP2754',
 'Item# CP7075',
 'Item# 554-76-RW-HBR',
 'Item# CP3012',
 'Item# CP6893',
 'Item# CP9603',
 'Item# CP8751',
 'Item# CP6958',
 'Item# CP8383',
 'Item# CP3677',
 'Item# CP1512',
 'Item# CP4183',
 'Item# CP2568',
 'Item# CP5343',
 'Item# CP54

### Nome do modelo

Seguindo para o nome do modelo das peças de xadrez, esta informação se encontra na primeira tag do tipo ```a``` dentro da tag ```div``` cujo atributo é ```class="item_sel_outer_mid_desc_title"```.

In [10]:
tag.find(name = 'div', class_ = "item_sel_outer_mid_desc_title").find(name = 'a').string

'Famous Staunton Wood Chess Pieces King Size 3" Golden Rosewood/Boxwood'

Fazendo isso com todos os itens.

In [11]:
ListaNomes = []

for subtag in tag.find_all(name = 'div', class_ = "item_sel_outer_mid_desc_title"):#.find(name = 'a'):
    ListaNomes.append(subtag.find(name='a').string)
    
ListaNomes

['Famous Staunton Wood Chess Pieces King Size 3" Golden Rosewood/Boxwood',
 'Wooden Unique Staunton Chess Pieces King Size 3-1/2" Golden Rosewood/Boxwood',
 'Wood Chess Set Pieces Monarch Staunton King Size 3" Golden Rosewood/Boxwood',
 'Chess Set Pieces Sovereign Staunton 3-1/4" Golden Rosewood/Boxwood',
 'Famous Staunton Wood Chess Pieces King Size 3-1/4" Golden Rosewood/Boxwood',
 'Wood Chess Set Pieces Monarch Staunton King Size 3" Ebonized/Boxwood',
 'Golden Rosewood/Boxwood Unique Staunton Wooden Chess Set Pieces King size 3" - Double Weighted',
 'Tournament Wood Chess Set Pieces Standard Staunton 3-3/4" Golden Rosewood/Boxwood',
 'Wood Chess Set Pieces Monarch Staunton King Size 3-1/2" Golden Rosewood/Boxwood',
 'Wood Chess Set Pieces Monarch Staunton King Size 3" Rosewood/Boxwood',
 'Bud Rosewood/Mahogany Wood Chess Set Pieces Galaxy Staunton 3" + 2 Extra Queens',
 'Mahogany/Boxwood Chess Set Pieces Galaxy Staunton 3" (76 mm) + 2 Extra Queens - Triple Weighted',
 'Golden Rosewo

### Preço das peças

Agora falta o preço dos itens, que estão na tag ```span``` e atributo ```class="sale_price"```.

In [12]:
tag.find(name='span', class_='sale_price').string

'Sale Price: $29.95'

Aplicando esse caminho em todos os produtos, temos

In [13]:
ListaPrecos = []

for subtag in tag.find_all(name='span', class_='sale_price'):
    ListaPrecos.append(subtag.string)
    
ListaPrecos

['Sale Price: $29.95',
 'Sale Price: $39.95',
 'Sale Price: $39.95',
 'Sale Price: $44.95',
 'Sale Price: $44.95',
 'Sale Price: $49.95',
 'Sale Price: $49.95',
 'Sale Price: $49.95',
 'Sale Price: $49.95',
 'Sale Price: $49.95',
 'Sale Price: $49.95',
 'Sale Price: $49.95',
 'Sale Price: $49.95',
 'Sale Price: $49.95',
 'Sale Price: $52.95',
 'Sale Price: $54.95',
 'Sale Price: $54.95',
 'Sale Price: $59.95',
 'Sale Price: $59.95',
 'Sale Price: $59.95',
 'Sale Price: $59.95',
 'Sale Price: $59.95',
 'Sale Price: $59.95',
 'Sale Price: $59.95',
 'Sale Price: $59.95',
 'Sale Price: $64.95',
 'Sale Price: $64.95',
 'Sale Price: $64.95',
 'Sale Price: $64.95',
 'Sale Price: $69.75',
 'Sale Price: $69.75',
 'Sale Price: $69.95',
 'Sale Price: $69.95',
 'Sale Price: $69.95',
 'Sale Price: $69.95',
 'Sale Price: $69.95',
 'Sale Price: $69.95',
 'Sale Price: $69.95',
 'Sale Price: $69.95',
 'Sale Price: $72.95',
 'Sale Price: $72.95',
 'Sale Price: $74.95',
 'Sale Price: $74.95',
 'Sale Pric

### Tipo de madeira

O tipo de material usado se encontra na tag ```strong``` cujo atributo ```style``` é ```"color:green"```.

In [30]:
tag.find(name = 'strong', style = 'color:green').string

'Golden Rosewood / Boxwood'

Fazendo isso para todos os produtos:

In [33]:
ListaMateriais = []

for subtag in tag.find_all(name = 'strong', style = 'color:green'):
    ListaMateriais.append(subtag.string)

Checando se coletamos corretamente o material da peça

In [34]:
ListaMateriais

['Golden Rosewood / Boxwood',
 'Golden Rosewood / Boxwood',
 'Golden Rosewood / Boxwood',
 'Golden Rosewood / Boxwood',
 'Golden Rosewood / Boxwood',
 'Ebonized / Boxwood',
 'Golden Rosewood / Boxwood',
 'Golden Rosewood / Boxwood',
 'Golden Rosewood / Boxwood',
 'Rosewood / Boxwood',
 'Bud Rosewood / Mahogany',
 'Mahogany / Boxwood',
 'Golden Rosewood / Boxwood',
 'Golden Rosewood / Boxwood',
 'Gold / Black Lacquered',
 'Golden Rosewood / Boxwood',
 'Golden Rosewood / Maple',
 'Ebonized / Boxwood',
 'Golden Rosewood / Boxwood',
 'Rosewood / Boxwood',
 'Golden Rosewood / Boxwood',
 'Golden Rosewood / Boxwood',
 'Golden Rosewood / Maple',
 'Wenge Wood / Boxwood',
 'Walnut / Boxwood',
 'Rosewoodnized / Boxwood',
 'Ebonized / Boxwood',
 'Rosewood / Boxwood',
 'Bud Rosewood / Boxwood',
 'Golden Rosewood / Boxwood',
 'Golden Rosewood / Boxwood',
 'Golden Rosewood / Boxwood',
 'Rosewood / Boxwood',
 'Golden Rosewood / Boxwood',
 'Ebonized / Boxwood',
 'Ebonized / Boxwood',
 'Ebonized / Boxwo

### Altura do Rei

Por fim, coletemos a altura do rei, que se encontra na tag ```strong```.

In [14]:
tag.find(name = 'strong', style='')

<strong>Size: 3"</strong>

Obs: eu coloquei como parâmetro ```style = ''``` pois há uma tag do mesmo tipo antes dela, que possui um atributo ```style='color:green'``` bem definido, enquanto que a informação que queremos está dentro da tag strong só que sem esse atributo.

Fazendo isso para todos os produtos:

In [15]:
ListaAlturas = []

for subtag in tag.find_all(name = 'strong', style = ''):
    ListaAlturas.append(subtag.string)
    
print(ListaAlturas[0])

Size: 3"


## Criação da Tabela

Feito a coleta de dados, criemos agora uma tabela com essas informações.

In [36]:
dicionario = {'Código':ListaCodigos, 'Nome':ListaNomes, 'Altura (Polegadas)':ListaAlturas, 'Material':ListaMateriais, 'Preço (USD)':ListaPrecos}

df = DataFrame(dicionario)

df

,Código,Nome,Altura (Polegadas),Material,Preço (USD)
0,Item# CP2485,"Famous Staunton Wood Chess Pieces King Size 3""...","Size: 3""",Golden Rosewood / Boxwood,Sale Price: $29.95
1,Item# CP1774,Wooden Unique Staunton Chess Pieces King Size ...,"Size: 3.5""",Golden Rosewood / Boxwood,Sale Price: $39.95
2,Item# CP3874,Wood Chess Set Pieces Monarch Staunton King Si...,"Size: 3""",Golden Rosewood / Boxwood,Sale Price: $39.95
3,Item# CP2358,"Chess Set Pieces Sovereign Staunton 3-1/4"" Gol...","Size: 3.25""",Golden Rosewood / Boxwood,Sale Price: $44.95
4,Item# CP2810,Famous Staunton Wood Chess Pieces King Size 3-...,"Size: 3.25""",Golden Rosewood / Boxwood,Sale Price: $44.95
...,...,...,...,...,...
160,Item# CP4483,Rosewood/Boxwood Chess Set Pieces Encore Staun...,"Size: 4.5""",Rosewood / Boxwood,Sale Price: $299.95
161,Item# CP4546,Bud Rosewood/Boxwood Chess Set Pieces Encore S...,"Size: 4.5""",Bud Rosewood / Boxwood,Sale Price: $349.95
162,Item# CP9768,Ebony/Boxwood Chess Set Pieces Encore Staunton...,"Size: 4.5""",Ebony / Boxwood,Sale Price: $349.95
163,Item# CP4983,Ebony/Bud Rosewood Chess Set Pieces Encore Sta...,"Size: 4.5""",Ebony Wood / Bud Rosewood,Sale Price: $399.95


## Limpeza dos dados

Criado a tabela, vejamos como a limpeza de cada coluna será feita. Usaremos o regex para tal e o site https://regex101.com/ para testar as expressões regulares.

In [17]:
from re import findall

Começando com a coluna de códigos, retiremos a parte 'Item# '.

In [37]:
regex = '[^Item# ].+'

df['Código'] = df['Código'].apply(lambda texto: findall(regex,texto)[0] )

Verificando a substituição.

In [21]:
df['Código']

0      CP2485
1      CP1774
2      CP3874
3      CP2358
4      CP2810
        ...  
160    CP4483
161    CP4546
162    CP9768
163    CP4983
164    CP2989
Name: Código, Length: 165, dtype: object

Agora vamos para o preço, que é basicamente tirar a parte 'Sale Price: $'. A regex que encontrei foi ```\d+\.\d+```

In [38]:
df['Preço (USD)'] = df['Preço (USD)'].apply(lambda texto: findall('\d+\.\d+', texto)[0] )

Verificando

In [26]:
df['Preço (USD)'].head()

0    29.95
1    39.95
2    39.95
3    44.95
4    44.95
Name: Preço (USD), dtype: object

Já para a altura, devemos retirar tanto a string 'Size: ', quanto o caractere final ". A regex que encontrei foi ```\d{1}\.?\d*```

In [39]:
df['Altura (Polegadas)'] = df['Altura (Polegadas)'].apply(lambda texto: findall('\d{1}\.?\d*', texto)[0] )

Verificando as alturas

In [28]:
df['Altura (Polegadas)']

0         3
1       3.5
2         3
3      3.25
4      3.25
       ... 
160     4.5
161     4.5
162     4.5
163     4.5
164     6.5
Name: Altura (Polegadas), Length: 165, dtype: object

## Conversão da tipagem

Feito a limpeza, vejamos qual a tipagem de cada coluna.

In [41]:
df.dtypes

Código                object
Nome                  object
Altura (Polegadas)    object
Material              object
Preço (USD)           object
dtype: object

Deste modo, precisamos converter a altura e o preço em ```float``` para realizar análises exploratórias.

In [43]:
df['Altura (Polegadas)'] = df['Altura (Polegadas)'].astype(float)

df['Preço (USD)'] = df['Preço (USD)'].astype(float)

Verificando se a conversão foi feita.

In [46]:
df.dtypes

Código                 object
Nome                   object
Altura (Polegadas)    float64
Material               object
Preço (USD)           float64
dtype: object